<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    npr Mini-Challenge 2 <br> 
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Si Ben Tran, Yannic Lais, Rami Tarabishi im HS 2023.<br>Bachelor of Science FHNW in Data Science.
</p>


# Aufgabe aus dem Space
Mini-Challenge 2-C (LE6):
Chatbot

You build a simple chatbot for hotel recommendation (here you can combine with npr and webscraping challenge) or extend the chatbot from the library (auxilio, please take contact with me), with rasa or dialogflow. You can also propose a use case. Important is the way intent are recognized and NER or variables are extracted. Delivery is a report on the use case, interactions and how intents are recognized and variables extracted. Also an error analysis on concrete conversations (confidence of models, explanation of predictions, etc.) should be undertaken. The submission can be done in a group of 2 or 3.


# Aufgabenstellung

Wir haben hier einen simplen ChatBot erstellt, welches die Anfrage des Benutzers entgegennimmt und eine Antwort zurückgibt. Die Anfrage des Benutzers wird durch unsere Preprocessing Pipeline bearbeitet und anschliessend durch ein Machine Learning Model (Kleines Neuronales Netz) klassifiziert - Intent Klassifikation. Anschliessend wird eine zufällige Antwort aus einer Liste von Antworten zurückgegeben. Weiter haben wir versucht die NER (Named Entity Recognition) zu implementieren, welche aber nicht so gut funktioniert hat.

# 1 Libraries

In [8]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
from tqdm import tqdm 

import nltk
# nltk.download('punkt')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import spacy
spacy_nlp = spacy.load("en_core_web_sm")

from src.preprocessing import tokenize, stem, bag_of_words
from src.news_scraper import scrape_srf_links, get_data_for_topics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocessing with bert Transformer
from transformers import BertTokenizer, BertModel


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 2 Create Data 

In [9]:
# Scrape srf webpage
url = "https://www.srf.ch"
topic_list = scrape_srf_links(url)
# create data 
get_data_for_topics(topic_list)

Combined JSON file 'data/intents_news.json' created successfully for all topics.


In [10]:
with open('data/intents_news.json', 'r') as f:
    intents_news = json.load(f)

with open('data/intents-scratch.json', 'r') as f:
    intents = json.load(f)

# combine intents_news with intens
intents["intents"].extend(intents_news["intents"])

# export intents formmated as json
with open('data/intents.json', 'w') as outfile:
    json.dump(intents, outfile, indent=4)

In [11]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents["intents"]:
    tag = intent["tag"]
    # add to tag list
    tags.append(tag)
    for pattern in intent["patterns"]:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

print(all_words)
print(tags)
print(xy)

['Hello', ',', 'my', 'name', 'is', 'Hi', ',', 'I', "'m", 'Hey', ',', 'I', 'go', 'by', 'I', 'am', 'Greetings', ',', 'call', 'me', 'You', 'can', 'call', 'me', 'They', 'call', 'me', 'My', 'name', "'s", 'I', "'d", 'like', 'to', 'introduce', 'myself', ',', 'I', "'m", 'Allow', 'me', 'to', 'introduce', 'myself', ',', 'I', 'am', 'I', 'go', 'by', 'the', 'name', 'People', 'know', 'me', 'as', 'Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'How', 'are', 'you', 'doing', 'Howdy', 'Hi', 'there', 'Hola', 'Good', 'morning', 'Good', 'afternoon', 'Good', 'evening', 'It', 'is', 'nice', 'to', 'meet', 'you', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Have', 'a', 'nice', 'day', 'See', 'you', '!', 'Later', '!', 'Goodbye', '!', 'I', "'m", 'off', '!', 'That', "'s", 'all', 'for', 'now', '.', 'See', 'you', 'soon', '!', 'I', "'ve", 'got', 'to', 'get', 'going', '.', 'Bye', '!', 'It', 'was', 'n

# 3 Bert Tokenizer

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Embedd all_words with bert
all_words_bert = []
for word in tqdm(all_words):
    tokenized_word = tokenizer.tokenize(word)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_word)
    all_words_bert.append(indexed_tokens)

print(all_words_bert)


100%|██████████| 1758/1758 [00:00<00:00, 17066.83it/s]

[[7592], [1010], [2026], [2171], [2003], [7632], [1010], [1045], [1005, 1049], [4931], [1010], [1045], [2175], [2011], [1045], [2572], [14806, 2015], [1010], [2655], [2033], [2017], [2064], [2655], [2033], [2027], [2655], [2033], [2026], [2171], [1005, 1055], [1045], [1005, 1040], [2066], [2000], [8970], [2870], [1010], [1045], [1005, 1049], [3499], [2033], [2000], [8970], [2870], [1010], [1045], [2572], [1045], [2175], [2011], [1996], [2171], [2111], [2113], [2033], [2004], [7632], [4931], [2129], [2024], [2017], [2003], [3087], [2045], [1029], [7592], [2204], [2154], [2054, 2015], [2039], [2129], [2024], [2017], [2725], [2129, 5149], [7632], [2045], [7570, 2721], [2204], [2851], [2204], [5027], [2204], [3944], [2009], [2003], [3835], [2000], [3113], [2017], [9061], [2156], [2017], [2101], [9119], [3835], [22331], [2000], [2017], [1010], [9061], [6229], [2279], [2051], [2031], [1037], [3835], [2154], [2156], [2017], [999], [2101], [999], [9119], [999], [1045], [1005, 1049], [2125], [9

# 4 Model

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')


# 5 Chatbot